<a href="https://colab.research.google.com/github/ShreyasKadiri/Machine_Learning/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Create a convolutional neural network and optimize it using keras tuner

In [12]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=ba2c1e88500d8152cdd961b63fe676529f0b489cda01837908159b5332677251
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=b0510da483b859f490413e0d6d4a03d689cde76695a3df6c0e6ca475d1aeec03
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

In [14]:
print(tf.__version__)

2.4.1


In [15]:
fashion_mnsit = keras.datasets.fashion_mnist

In [16]:
(train_images, train_labels), (test_images, test_lables) = fashion_mnsit.load_data()

In [17]:
train_images=train_images/255.0
test_images=test_images/255.0

In [18]:
train_images[0].shape

(28, 28)

In [19]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [20]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [21]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='Mnist Fashion with Keras Tuner')

In [23]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.9111666679382324

Best val_accuracy So Far: 0.9111666679382324
Total elapsed time: 00h 01m 42s
INFO:tensorflow:Oracle triggered exit


In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2973824   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,997,146
Trainable params: 2,997,146
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1896 - accuracy: 0.9289 - val_loss: 0.2444 - val_accuracy: 0.9112
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1387 - accuracy: 0.9486 - val_loss: 0.2568 - val_accuracy: 0.9110
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0999 - accuracy: 0.9623 - val_loss: 0.2996 - val_accuracy: 0.9135
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0718 - accuracy: 0.9731 - val_loss: 0.3389 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0513 - accuracy: 0.9813 - val_loss: 0.3575 - val_accuracy: 0.9137
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0363 - accuracy: 0.9869 - val_loss: 0.4263 - val_accuracy: 0.9122
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0365 - accuracy: 0.9881 - val_loss: 0.4660 - val_accuracy